In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.metrics import confusion_matrix

PATH = r'SMSSpamCollection.txt'

col_names = ['spam', 'text']
df_raw = pd.read_csv(PATH, sep='\t', names=col_names)
df = pd.read_csv(PATH, sep='\t', names=col_names)

In [2]:
def clean_text(x):
    x = x.lower()
    x = x.replace('.','')
    x = x.replace(',','')
    x = x.replace(':','')
    x = x.replace(';','')
    x = x.replace('!','')
    x = x.replace('?','')
    x = x.replace('<br>','')
    x = x.replace('<br />','')
    x = ' '.join([word for word in x.split() if word.isalpha()])
    return x

df['text'] = df['text'].apply(clean_text)
df['spam'].replace({"ham": 0, "spam": 1}, inplace=True)

In [3]:
df['text'] = df['text'].apply(clean_text)
df['spam'].replace({"ham": 0, "spam": 1}, inplace=True)

In [4]:
df

,spam,text
0,0,go until jurong point crazy available only in ...
1,0,ok lar joking wif u oni
2,1,free entry in a wkly comp to win fa cup final ...
3,0,u dun say so early hor u c already then say
4,0,nah i think he goes to usf he lives around her...
...,...,...
5567,1,this is the time we have tried contact u u hav...
5568,0,will ü b going to esplanade fr home
5569,0,pity was in mood for that soany other suggestions
5570,0,the guy did some bitching but i acted like be ...


In [5]:
tfidf = TfidfVectorizer(min_df=10)
inputs = tfidf.fit_transform(df['text'])
tfidf

TfidfVectorizer(min_df=10)

In [6]:
tfidf_df = pd.DataFrame(inputs.todense(), columns = tfidf.get_feature_names())
tfidf_df.head(10)

,abiola,able,about,abt,account,across,actually,address,admirer,aft,...,yes,yesterday,yet,yo,you,your,yours,yourself,yr,yup
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.108801,0.000000,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.478202,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.097003,0.000000,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.120464,0.0,0.0,0.0,0.0


In [7]:

target = df['spam']

train_input,test_input,train_target,test_target = train_test_split(inputs,target, test_size=0.25)

model = LogisticRegression()
model.fit(train_input,train_target)

predictions = model.predict(test_input)


crosstab = pd.crosstab(test_target, predictions, rownames = ['Actual'], colnames =['Predicted'], margins = True)
print('Cross table:')
print(crosstab)
print('\n')

print("Classification report:")
print(classification_report(test_target, predictions))

print(f"Spam detection precision: {round(precision_score(test_target,predictions),4)}")
print(f"Spam detection recall: {round(recall_score(test_target,predictions),4)}")
print(f"No-spam detection precision:: {round(precision_score(test_target,predictions,pos_label = 0),4)}")
print(f"No-spam detection recall:: {round(recall_score(test_target,predictions,pos_label = 0),4)}")
print(f"Classification accuracy: {round(accuracy_score(test_target,predictions),4)}")

Cross table:
Predicted     0    1   All
Actual                    
0          1193    6  1199
1            47  147   194
All        1240  153  1393


Classification report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      1199
           1       0.96      0.76      0.85       194

    accuracy                           0.96      1393
   macro avg       0.96      0.88      0.91      1393
weighted avg       0.96      0.96      0.96      1393

Spam detection precision: 0.9608
Spam detection recall: 0.7577
No-spam detection precision:: 0.9621
No-spam detection recall:: 0.995
Classification accuracy: 0.962


In [8]:
weights = list(zip(tfidf.get_feature_names(), model.coef_[0]))
weights.sort(key = lambda x:x[1])

In [9]:
weights_dict = dict(weights)
weights_dict

{'me': -2.4843179296166955,
 'my': -2.125869854480259,
 'later': -1.663829114778846,
 'come': -1.4984563148888868,
 'ok': -1.4824560050376951,
 'sorry': -1.304893840504233,
 'am': -1.1924306946447667,
 'going': -1.115746121796969,
 'home': -1.1021971212428403,
 'it': -1.0749279846618425,
 'that': -1.0603494681293766,
 'but': -1.0499067141597584,
 'its': -1.0425821189159798,
 'up': -0.9880267299628718,
 'so': -0.9734447984148407,
 'when': -0.9364140107846383,
 'in': -0.916111071452728,
 'he': -0.9124120786385803,
 'da': -0.9083703546669273,
 'lor': -0.8772169022073015,
 'got': -0.8740037009404028,
 'then': -0.8677065225279661,
 'way': -0.8520798461581847,
 'work': -0.8388614683032826,
 'sir': -0.8371932608998627,
 'well': -0.8253023544877516,
 'can': -0.8183019530389498,
 'at': -0.8144034102936987,
 'not': -0.7988856379989863,
 'still': -0.7900731809247804,
 'she': -0.7848869392345252,
 'how': -0.7763475619286531,
 'do': -0.7567747482869671,
 'lol': -0.7565200011457764,
 'ask': -0.75523

In [10]:
def count_sentiment(row):
    sent_value = 0
    row_dct = row.to_dict()
    for key, value in row_dct.items():
        if value != 0:
            sent_value += weights_dict[key]
    return sent_value

In [11]:
df['sentiment_score'] = tfidf_df.apply(count_sentiment, axis=1)

In [12]:
df.sort_values('sentiment_score', ascending=False)

,spam,text,sentiment_score
673,1,get ur ringtone free now reply to this msg wit...,36.146277
3999,1,we tried to call you re your reply to our sms ...,32.469963
583,1,we tried to contact you re your reply to our o...,30.885722
4091,1,we tried to call you re your reply to our sms ...,30.344716
5012,1,you have won a guaranteed cash or a prize to c...,30.133227
...,...,...,...
4418,0,says that quitting at a day so i take much not...,-11.121751
1085,0,for me the love should start with attractioni ...,-11.323127
31,0,yeah he got in at and was v apologetic n had f...,-11.657967
5139,0,oh shut it omg yesterday i had a dream that i ...,-12.070015


In [13]:
import spacy
nlp = spacy.load("en_core_web_sm")
df_raw.head(10)

,spam,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [14]:
modeled_text = nlp(df_raw.iloc[-1][1])
modeled_text

Rofl. Its true to its name

In [15]:
[token.text for token in modeled_text]

['Rofl', '.', 'Its', 'true', 'to', 'its', 'name']

In [16]:
print([token.lower_ for token in modeled_text])
print([token.lower for token in modeled_text])
print([token.lemma_ for token in modeled_text])
print([token.pos_ for token in modeled_text])

['rofl', '.', 'its', 'true', 'to', 'its', 'name']
[10885026304460510830, 12646065887601541794, 12513610393978129441, 7434368892455186804, 3791531372978436496, 12513610393978129441, 18309932012808971453]
['Rofl', '.', 'its', 'true', 'to', 'its', 'name']
['PROPN', 'PUNCT', 'PRON', 'ADJ', 'ADP', 'PRON', 'NOUN']


In [17]:
from __future__ import unicode_literals
from collections import Counter
advs = []
nouns = []
verbs = []
summ = []

for i in range(len(df_raw)):
    modeled_text_df = nlp(df_raw.iloc[i][1])
    c = Counter(([token.pos_ for token in modeled_text_df]))
    advs.append(c['ADV'])
    nouns.append(c['NOUN'])
    verbs.append(c['VERB'])
    summ.append(c['ADV']+c['NOUN']+c['VERB'])

In [18]:
df_raw['SUM'] = summ
df_raw['ADVS'] = advs
df_raw['ADVS_%'] = round(100*df_raw['ADVS'] / df_raw['SUM'])
df_raw['NOUNS'] = nouns
df_raw['NOUNS_%'] = round(100*df_raw['NOUNS'] / df_raw['SUM'])
df_raw['VERBS'] = verbs
df_raw['VERBS_%'] = round(100*df_raw['VERBS'] / df_raw['SUM'])
df_raw

,spam,text,SUM,ADVS,ADVS_%,NOUNS,NOUNS_%,VERBS,VERBS_%
0,ham,"Go until jurong point, crazy.. Available only ...",8,2,25.0,4,50.0,2,25.0
1,ham,Ok lar... Joking wif u oni...,3,0,0.0,2,67.0,1,33.0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,11,0,0.0,6,55.0,5,45.0
3,ham,U dun say so early hor... U c already then say...,7,3,43.0,1,14.0,3,43.0
4,ham,"Nah I don't think he goes to usf, he lives aro...",7,3,43.0,0,0.0,4,57.0
...,...,...,...,...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,11,1,9.0,7,64.0,3,27.0
5568,ham,Will ü b going to esplanade fr home?,5,0,0.0,2,40.0,3,60.0
5569,ham,"Pity, * was in mood for that. So...any other s...",4,1,25.0,3,75.0,0,0.0
5570,ham,The guy did some bitching but I acted like i'd...,8,1,12.0,3,38.0,4,50.0


In [19]:
tfidf_df['CLASS'] = df['spam']
tfidf_df

,abiola,able,about,abt,account,across,actually,address,admirer,aft,...,yesterday,yet,yo,you,your,yours,yourself,yr,yup,CLASS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [20]:
# KNN

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

X = tfidf_df.drop(labels=['CLASS'], axis=1)
Y = tfidf_df['CLASS']

steps = [('scaler', StandardScaler()), ('KNN', KNeighborsClassifier())]
pipeline = Pipeline(steps)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)
knn_scaled_untuned = pipeline.fit(X_train, y_train)
knn_predict = knn_scaled_untuned.predict(X_test)

print("KNN (scaled, untuned) classification report:")
print(classification_report(y_test, knn_predict))
print(f"KNN (scaled, untuned) accuracy: {round(pipeline.score(X_test, y_test), 4)*100}%")

KNN (scaled, untuned) classification report:
              precision    recall  f1-score   support

           0       0.94      0.99      0.97       966
           1       0.93      0.60      0.73       149

    accuracy                           0.94      1115
   macro avg       0.93      0.80      0.85      1115
weighted avg       0.94      0.94      0.94      1115

KNN (scaled, untuned) accuracy: 94.08%


In [21]:
# SVC
from sklearn.svm import SVC

X = tfidf_df.drop(labels=['CLASS'], axis=1)
Y = tfidf_df['CLASS']

steps = [('scaler', StandardScaler()), ('SVM', SVC())]
pipeline = Pipeline(steps)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=21)

svc_scaled_untuned = pipeline.fit(X_train, y_train)
svc_pred = svc_scaled_untuned.predict(X_test)

print("SVC (scaled, untuned) classification report:")
print(classification_report(y_test, svc_pred))
print(f'SVC (scaled, untuned) accuracy: {round(svc_scaled_untuned.score(X_test, y_test), 2)*100}%')

SVC (scaled, untuned) classification report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       960
           1       1.00      0.86      0.92       155

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.96      1115
weighted avg       0.98      0.98      0.98      1115

SVC (scaled, untuned) accuracy: 98.0%


In [22]:
# KMEANS
from sklearn.cluster import KMeans

X = tfidf_df.drop(labels=['CLASS'], axis=1)
Y = tfidf_df['CLASS']

steps = [('scaler', StandardScaler()), ('KMEANS', KMeans(2))]
pipeline = Pipeline(steps)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)

kmeans_scaled_untuned = pipeline.fit(X_train)
kmeans_pred = kmeans_scaled_untuned.predict(X_test)

print("KMeans (scaled, untuned) classification report:")
print(classification_report(y_test, kmeans_pred))

KMeans (scaled, untuned) classification report:
              precision    recall  f1-score   support

           0       0.02      0.00      0.00       966
           1       0.04      0.23      0.06       149

    accuracy                           0.03      1115
   macro avg       0.03      0.12      0.03      1115
weighted avg       0.02      0.03      0.01      1115

